In [1]:
import multiprocessing
multiprocessing.set_start_method("fork")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm 

feat_ml_df = pd.read_pickle("data/deleteme.pkl")#.sort_values("Date")
gender_df = pd.read_csv("data/fighter_genders.csv")
feat_ml_df2 = feat_ml_df.merge(gender_df, on="FighterID", how="left")
feat_ml_df2 = feat_ml_df2.sort_values("Date")
# feat_ml_df2["gender"].isnull().any()
feat_ml_df.head()

,FighterID,OpponentID,oldFighterEloAD,oldOpponentEloAD,predTargetAD,targetAD,Date,newFighterEloAD,newOpponentEloAD,oldEloDiffAD,...,FighterOpen,OpponentOpen,FighterCloseLeft,FighterCloseRight,OpponentCloseLeft,OpponentCloseRight,p_fighter,p_opponent,p_fighter_midpoint,p_fighter_implied
0,2335629/bj-penn,2335885/jens-pulver,0.842355,0.364097,0.478257,2.645751,2007-06-23,1.275853,-0.069401,0.478257,...,-490.0,340.0,-357.0,-330.0,225.0,280.0,0.830508,0.227273,0.801618,0.785142
1,2335676/joe-lauzon,2354360/brandon-melendez,0.091024,0.000000,0.091024,2.000000,2007-06-23,0.472819,-0.381795,0.091024,...,-430.0,330.0,-700.0,-475.0,355.0,425.0,0.811321,0.232558,0.789381,0.777217
2,2335447/anderson-silva,2335475/nate-marquardt,0.227196,0.736307,-0.509111,-1.000000,2007-07-07,0.129018,0.834485,-0.509111,...,-170.0,150.0,-145.0,-145.0,125.0,125.0,0.629630,0.400000,0.614815,0.611511
3,2335302/heath-herring,2335521/antonio-rodrigo-nogueira,0.106410,0.000000,0.106410,-2.645751,2007-07-07,-0.444022,0.550432,0.106410,...,470.0,-810.0,500.0,500.0,-700.0,-700.0,0.175439,0.890110,0.142664,0.164646
4,2335694/frankie-edgar,2335717/mark-bocek,0.000000,0.000000,0.000000,1.414214,2007-07-07,0.282843,-0.282843,0.000000,...,-260.0,180.0,-230.0,-230.0,190.0,190.0,0.722222,0.357143,0.682540,0.669118


In [2]:
feat_cols = [
    "oldEloDiffordinal_fighter_result", 
    "oldEloDiffsubmission_fighter_result",
    "oldEloDifftko_ko_fighter_result",
    "oldEloDiffdecision_fighter_result",
#     "oldEloDifffinish_fighter_result",
#     "oldEloDifffighter_result_time_left",
    
    'oldEloDiffAD', 
    'oldEloDiffADTB', 
    'oldEloDiffADTM', 
    'oldEloDiffKD',
    'oldEloDiffRV', 
    'oldEloDiffSCBL', 
    'oldEloDiffSCHL', 
    'oldEloDiffSGBL',
    'oldEloDiffSGHL', 
    'oldEloDiffSM', 'oldEloDiffSSL', 'oldEloDiffTDL',
    'oldEloDiffTDS', 
    'oldEloDiffTD_fails', 
    'oldEloDiffTSL',
    'oldEloDiffclinch_strikes_landed', 
    'oldEloDiffdistance_strikes_landed',
    'oldEloDiffground_strikes_landed', 
    'oldEloDiffstanding_strikes',
    
    'oldEloDiffWin',
    
    "t_since_last_fight_log_diff", 
#     "t_since_last_fight_diff",
    "total_fights_sqrt_diff", 
    "total_ufc_fights_diff",
    
    "age_diff", "reach_diff", 
#     "weight_diff", 
    "log_weight_diff",
    "height_diff",
]

In [3]:
from model.mma_hier_log_reg_stan import HierPcaSymmetricModel

hier_model = HierPcaSymmetricModel(feat_cols, n_pca=8, 
                                   beta_prior_std=1.0, intra_group_std=0.1, 
                                   mcmc=False)

In [5]:
from model_selection.cross_val_pipeline import TimeSeriesCrossVal

temp_df = feat_ml_df2.dropna(subset=[*feat_cols, "targetWin"])
tscv = TimeSeriesCrossVal(n_splits=4)
fold_pred_df = tscv.get_cross_val_preds(hier_model, temp_df)
hier_metrics = tscv.score_preds()
hier_metrics

training on date range: 2007-06-23 2012-04-14
Initial log joint probability = -2531.48
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      16      -322.154   0.000239829     0.0049665           1           1       20   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
training on date range: 2007-06-23 2015-04-04
Initial log joint probability = -2618.73
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      16      -613.242   5.77806e-05    0.00853694      0.3421           1       19   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
training on date range: 2007-06-23 2017-11-18
Initial log joint probability = -2533.3
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      14      -979.579   0.000325463     0.0101634           1           1    

,test_fold,min_test_date,max_test_date,n_test_days,n_test_fights,log_loss,accuracy_score,ml_log_loss
0,1,2012-04-21,2015-04-04,108,478,0.624513,0.631799,0.613181
1,2,2015-04-11,2017-11-18,108,557,0.660575,0.588869,0.661171
2,3,2017-11-25,2020-02-15,108,750,0.640064,0.626667,0.642069
3,4,2020-02-22,2022-03-19,107,891,0.624976,0.635241,0.628963


In [ ]:
temp_df["p_fighter_midpoint"]

In [7]:
from model.mma_hier_log_reg_stan import logit

class MidpointHier(HierPcaSymmetricModel):
    
    def fit_predict(self, train_df, test_df, feat_cols=None):
        if self.stan_model is None:
            self._load_stan_model()
        if not feat_cols:
            feat_cols = self.feat_cols
        scale_ = (train_df[feat_cols]**2).mean(0)
        self.scale_ = scale_
        X_train = train_df[feat_cols] / scale_
        X_test = test_df[feat_cols] / scale_
        
        X_pca_train = self.pca.fit_transform(X_train)
        X_pca_test = self.pca.transform(X_test)

        y_train = train_df["targetWin"]
        y_test = test_df["targetWin"]

        ml_train = logit(train_df["p_fighter_midpoint"])
        ml_test = logit(test_df["p_fighter_midpoint"])
        
        is_m_train = train_df["gender"].map({"M":1, "W":0})
        is_m_test = test_df["gender"].map({"M":1, "W":0})
        
        data = {
            "n": train_df.shape[0],
            "n2": test_df.shape[0],
            "d": self.n_pca,
            "y": y_train.astype(int).values,
            "beta_prior_std": self.beta_prior_std,
            "intra_group_std": self.intra_group_std,
            "is_m": is_m_train.values,
            "is_m2": is_m_test.values,
            "X": X_pca_train,
            "ml_logit": ml_train.values,
            "X2": X_pca_test,
            "ml_logit2": ml_test.values,
        }
        if self.mcmc:
            fit = self._fit_mcmc(data)
            return fit["y_pred"].mean(0) 
        fit = self._fit_opt(data)
        return fit["y_pred"]
    
mid_model = MidpointHier(feat_cols, n_pca=8, 
                                   beta_prior_std=1.0, intra_group_std=0.1, 
                                   mcmc=False)
fold_pred_df = tscv.get_cross_val_preds(mid_model, temp_df)
mid_metrics = tscv.score_preds()
mid_metrics

training on date range: 2007-06-23 2012-04-14
Initial log joint probability = -1730.79
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      15      -322.119   0.000110931    0.00465868      0.4448      0.8692       18   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
training on date range: 2007-06-23 2015-04-04
Initial log joint probability = -1946.92
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      14      -612.169   0.000134221     0.0106726           1           1       16   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
training on date range: 2007-06-23 2017-11-18
Initial log joint probability = -3096.5
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      15      -979.224   0.000199284     0.0254599           1           1    

,test_fold,min_test_date,max_test_date,n_test_days,n_test_fights,log_loss,accuracy_score,ml_log_loss
0,1,2012-04-21,2015-04-04,108,478,0.623385,0.629707,0.613181
1,2,2015-04-11,2017-11-18,108,557,0.662083,0.587074,0.661171
2,3,2017-11-25,2020-02-15,108,750,0.640476,0.624000,0.642069
3,4,2020-02-22,2022-03-19,107,891,0.624473,0.640853,0.628963


In [12]:
(mid_metrics["log_loss"] / hier_metrics["log_loss"]).mean()

1.00007909260828

# Okay, using the midpoint does not make much difference. If anything makes it worse

# Welp try a naive train/test split then

In [17]:
max_train_date = pd.to_datetime("2021-07-01")
train_df = feat_ml_df2.query(f"Date <= '{max_train_date}'").dropna(subset=feat_cols)
test_df  = feat_ml_df2.query(f"Date > '{max_train_date}'").dropna(subset=feat_cols)
train_df.shape, test_df.shape

((2888, 218), (331, 218))

In [19]:
y_pred = hier_model.fit_predict(train_df, test_df)
pred_df = test_df.assign(y_pred=y_pred)
pred_df.head()

Initial log joint probability = -5280.96


,FighterID,OpponentID,oldFighterEloAD,oldOpponentEloAD,predTargetAD,targetAD,Date,newFighterEloAD,newOpponentEloAD,oldEloDiffAD,...,FighterCloseLeft,FighterCloseRight,OpponentCloseLeft,OpponentCloseRight,p_fighter,p_opponent,p_fighter_midpoint,p_fighter_implied,gender,y_pred
2897,4077451/niko-price,4418962/michel-pereira,-0.110615,-0.007896,-0.102719,0.0,2021-07-10,-0.090071,-0.028440,-0.102719,...,140.0,180.0,-235.0,-175.0,0.400000,0.636364,0.381818,0.385965,M,0.282237
2896,3956260/ryan-hall,4350812/ilia-topuria,0.327320,0.538227,-0.210906,0.0,2021-07-10,0.369502,0.496045,-0.210906,...,155.0,185.0,-235.0,-209.0,0.350877,0.687500,0.331689,0.337909,M,0.386529
2895,4222369/jerome-rivera,4321051/zhalgas-zhumagulov,-0.164644,-0.351310,0.186667,0.0,2021-07-10,-0.201977,-0.313977,0.186667,...,172.0,250.0,-333.0,-220.0,0.250000,0.800000,0.225000,0.238095,M,0.222928
2893,2220951/greg-hardy,3722422/tai-tuivasa,0.069877,-0.392619,0.462495,0.0,2021-07-10,-0.022622,-0.300120,0.462495,...,-110.0,105.0,-133.0,-110.0,0.408163,0.629630,0.389267,0.393299,M,0.390775
2894,3166126/dricus-du-plessis,4024732/trevin-giles,0.000000,0.238362,-0.238362,0.0,2021-07-10,0.047672,0.190690,-0.238362,...,-125.0,-104.0,-122.0,100.0,0.476190,0.565217,0.455487,0.457256,M,0.400939


    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      15      -1819.46   0.000257576      0.011597           1           1       17   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


In [26]:
from sklearn.metrics import accuracy_score, log_loss

log_loss(y_true=pred_df["targetWin"], y_pred=pred_df["y_pred"])

0.6119004325698266

In [27]:
log_loss(y_true=pred_df["targetWin"], y_pred=pred_df["p_fighter_implied"])

0.6169001683494999

['t_since_last_fight_log',
 'total_fights_sqrt',
 'total_ufc_fights',
 'age',
 'reach',
 'log_weight',
 'height']

In [45]:
elo_suffixes = [
    col[len("oldEloDiff"):] for col in feat_cols if col.startswith("oldEloDiff")
]
elo_opponent_cols = [f"oldOpponentElo{suff}" for suff in elo_suffixes]
elo_fighter_cols = [f"oldFighterElo{suff}" for suff in elo_suffixes]

real_prefixes = [
    't_since_last_fight',
    'total_fights',
    'total_ufc_fights',
    'age',
    'imp_reach', 
    'imp_weight',
    'imp_height',
]

# real_prefixes = [
#     col[:-len("_diff")] for col in feat_cols if col.endswith("diff")
# ]
real_opponent_cols = [f"{pref}_opp" for pref in real_prefixes]
real_fighter_cols = real_prefixes

In [46]:
feat_ml_df2[elo_opponent_cols + elo_fighter_cols + real_opponent_cols + real_fighter_cols]

,oldOpponentEloordinal_fighter_result,oldOpponentElosubmission_fighter_result,oldOpponentElotko_ko_fighter_result,oldOpponentElodecision_fighter_result,oldOpponentEloAD,oldOpponentEloADTB,oldOpponentEloADTM,oldOpponentEloKD,oldOpponentEloRV,oldOpponentEloSCBL,...,imp_reach_opp,imp_weight_opp,imp_height_opp,t_since_last_fight,total_fights,total_ufc_fights,age,imp_reach,imp_weight,imp_height
0,1.026746,0.238552,0.250160,0.049322,0.364097,0.036204,0.046619,-0.086080,0.320000,0.231854,...,70.000000,135.393229,68.299203,273,15,10,28.545205,70.0,155.000000,69.000000
1,-0.265507,-0.079613,-0.020184,-0.066127,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,71.634058,167.003261,69.868379,3,16,1,23.101370,71.3,155.000000,70.000000
2,1.876177,0.567532,0.146170,0.448772,0.736307,0.152549,0.018275,0.428642,0.178640,0.466412,...,74.000000,185.562500,71.886497,154,23,3,32.252055,77.0,185.000000,74.000000
3,2.431473,0.781250,0.286725,0.295523,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,77.000000,249.832618,74.838444,91,32,2,29.367123,78.0,249.832618,75.612176
4,1.170114,0.501547,0.110505,-0.053990,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,75.000000,155.221258,72.362459,154,6,1,25.739726,68.0,136.000000,66.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3214,2.424475,0.279215,0.781818,0.302147,0.131944,-0.005186,0.045079,0.366734,0.006129,-0.222682,...,76.000000,235.000000,75.000000,182,19,14,31.120548,80.0,261.000000,76.000000
3215,2.192568,0.555066,0.311679,0.459076,0.006633,-0.159462,0.002672,0.000275,0.188348,-0.315380,...,68.000000,126.000000,65.000000,168,21,5,29.235616,66.0,125.000000,66.000000
3216,2.023056,0.209947,0.658022,0.287118,0.025992,0.017424,0.040854,0.156362,-0.003062,0.351847,...,65.000000,125.000000,67.000000,70,28,8,33.490411,64.0,125.000000,64.000000
3217,1.014924,0.072868,0.267804,0.333578,0.000000,0.000000,0.000000,0.040000,0.000000,-0.413868,...,70.000000,148.000000,70.000000,217,22,2,31.180822,73.0,146.000000,71.000000


In [49]:
feat_ml_df2.dropna(subset=[*real_fighter_cols, *real_opponent_cols]).shape

(3134, 218)

In [132]:
import torch
from sklearn.utils import shuffle, gen_batches
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tqdm import tqdm
        

class SymmetricModel(torch.nn.Module):
    
    def __init__(self, input_dim, width_1, width_2):
        super().__init__()
        self.linear1 = torch.nn.Linear(input_dim, width_1)
        self.activation = torch.nn.Tanh() # tanh(x) = -tanh(-x)
        self.linear2 = torch.nn.Linear(width_1, width_2)
        # take diff
        self.linear3 = torch.nn.Linear(width_2, 1, bias=False)
        self.softmax = torch.nn.Sigmoid()
        
    def forward(self, ml_logit, x_f, x_o):
        x_f = self.linear1(x_f)
        x_f = self.activation(x_f)
        x_f = self.linear2(x_f)
        x_f = self.activation(x_f)
        
        x_o = self.linear1(x_o)
        x_o = self.activation(x_o)
        x_o = self.linear2(x_o)
        x_o = self.activation(x_o)
        
        x_diffs = x_f - x_o
        x_diffs = self.linear3(x_diffs)
        y_hat = self.softmax(ml_logit + x_diffs)
        return y_hat
    
class SymmetricModelWrapper(object):
    
    def __init__(self, fighter_cols, opponent_cols, n_pca=16, 
                 width_1=16, width_2=16, lr=0.001, n_epochs=30, batch_size=64):
        self.fighter_cols = fighter_cols
        self.opponent_cols = opponent_cols
        assert len(fighter_cols) == len(opponent_cols) # fuck style
        self.n_pca = n_pca
        self.width_1 = width_1
        self.width_2 = width_2
        self.lr = lr
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self._model = None
        
    def _get_X(self, train_df, test_df):
        X_temp = np.concatenate([train_df[self.fighter_cols], 
                                 train_df[self.opponent_cols]])
        scale_ = (X_temp**2).mean(0)
        pca = PCA(n_components=self.n_pca, whiten=True)
        pca.fit(X_temp / scale_)
        
        X_f_train = pca.transform(train_df[self.fighter_cols].values / scale_)
        X_o_train = pca.transform(train_df[self.opponent_cols].values / scale_)
        
        X_f_test = pca.transform(test_df[self.fighter_cols].values / scale_)
        X_o_test = pca.transform(test_df[self.opponent_cols].values / scale_)
        
        return X_f_train, X_o_train, X_f_test, X_o_test
    
    def _train(self, X_f_train, X_o_train, y_train, ml_logit_train):
        input_dim = X_f_train.shape[1]
        self._model = SymmetricModel(input_dim, self.width_1, self.width_2)
        optimizer = torch.optim.Adam(self._model.parameters(), lr=self.lr)
        loss_fn = torch.nn.BCELoss() # binary cross entropy
        for epoch in tqdm(range(self.n_epochs)):
            # shuffle dataset
            shuffle_inds = shuffle(range(X_f_train.shape[0]))
            X_f = torch.Tensor(X_f_train[shuffle_inds,:])
            X_o = torch.Tensor(X_o_train[shuffle_inds,:])
            ml = torch.Tensor(ml_logit_train[shuffle_inds]).view(-1,1)
            y = torch.Tensor(y_train[shuffle_inds]).view(-1,1)
            # iterate over batches
            for batch_inds in gen_batches(X_f.shape[0], self.batch_size):
                optimizer.zero_grad()
                y_hat = self._model(ml[batch_inds], X_f[batch_inds,:], X_o[batch_inds,:])
                loss = loss_fn(y_hat, y[batch_inds])
                loss.backward()
                optimizer.step()
        
    def fit_predict(self, train_df, test_df):
        X_f_train, X_o_train, X_f_test, X_o_test = self._get_X(train_df, test_df)
        
        y_train = train_df["targetWin"].values
        y_test = test_df["targetWin"].values

        ml_logit_train = logit(train_df["p_fighter_implied"]).values
        ml_logit_test = logit(test_df["p_fighter_implied"]).values
        
        self._train(X_f_train, X_o_train, y_train, ml_logit_train)
        
        y_hat = self._model(
            torch.Tensor(ml_logit_test).view(-1,1), 
            torch.Tensor(X_f_test),
            torch.Tensor(X_o_test),
        )
        return y_hat.detach().numpy()[:,0]
            
# huh, this means i can include a feature for gender or weight class
torch_mod = SymmetricModelWrapper(
    fighter_cols=(elo_fighter_cols+real_fighter_cols),
    opponent_cols=(elo_opponent_cols+real_opponent_cols),
    n_pca=8,
    n_epochs=10,
)
torch_mod

In [133]:
subset_cols = torch_mod.fighter_cols + torch_mod.opponent_cols
train_df = feat_ml_df2.query(f"Date <= '{max_train_date}'").dropna(subset=subset_cols)
test_df = feat_ml_df2.query(f"Date > '{max_train_date}'").dropna(subset=subset_cols)
train_df.shape, test_df.shape

((2803, 218), (331, 218))

In [134]:
y_pred = torch_mod.fit_predict(train_df, test_df)
log_loss(y_pred=y_pred, y_true=test_df["targetWin"])

100%|███████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 39.34it/s]


0.6132680034979593

In [135]:
log_loss(y_pred=test_df["p_fighter_implied"], y_true=test_df["targetWin"])

0.6169001683494999

In [138]:
torch_mod.lr

0.001

In [139]:
torch_df = feat_ml_df2.dropna(subset=subset_cols)

torch_mod2 = SymmetricModelWrapper(
    fighter_cols=(elo_fighter_cols+real_fighter_cols),
    opponent_cols=(elo_opponent_cols+real_opponent_cols),
    n_pca=16,
    lr=0.0005,
    n_epochs=10,
)

tscv = TimeSeriesCrossVal(n_splits=4)
tscv.get_cross_val_preds(torch_mod2, torch_df)
torch_metrics = tscv.score_preds()
torch_metrics

training on date range: 2007-06-23 2012-03-02


100%|██████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 167.60it/s]


training on date range: 2007-06-23 2015-03-14


100%|██████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 105.48it/s]

training on date range: 2007-06-23 2017-10-28



100%|███████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 72.95it/s]


training on date range: 2007-06-23 2020-01-25


100%|███████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 49.46it/s]


,test_fold,min_test_date,max_test_date,n_test_days,n_test_fights,log_loss,accuracy_score,ml_log_loss
0,1,2012-04-14,2015-03-14,107,458,0.618871,0.648472,0.614366
1,2,2015-03-21,2017-10-28,107,547,0.658428,0.606947,0.661040
2,3,2017-11-04,2020-01-25,107,748,0.642268,0.620321,0.642368
3,4,2020-02-08,2022-03-05,107,886,0.627927,0.646727,0.631548


In [141]:
(torch_metrics["log_loss"] / torch_metrics["ml_log_loss"]).mean()

0.999373394086803

In [142]:
torch_mod3 = SymmetricModelWrapper(
    fighter_cols=(elo_fighter_cols+real_fighter_cols),
    opponent_cols=(elo_opponent_cols+real_opponent_cols),
    n_pca=16,
    lr=0.0005,
    n_epochs=20,
)

tscv.get_cross_val_preds(torch_mod3, torch_df)
torch_metrics3 = tscv.score_preds()
torch_metrics3

training on date range: 2007-06-23 2012-03-02


100%|██████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 194.87it/s]


training on date range: 2007-06-23 2015-03-14


100%|██████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 116.30it/s]


training on date range: 2007-06-23 2017-10-28


100%|███████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 74.97it/s]


training on date range: 2007-06-23 2020-01-25


100%|███████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 47.04it/s]


,test_fold,min_test_date,max_test_date,n_test_days,n_test_fights,log_loss,accuracy_score,ml_log_loss
0,1,2012-04-14,2015-03-14,107,458,0.614928,0.641921,0.614366
1,2,2015-03-21,2017-10-28,107,547,0.664068,0.597806,0.661040
2,3,2017-11-04,2020-01-25,107,748,0.643283,0.631016,0.642368
3,4,2020-02-08,2022-03-05,107,886,0.628267,0.639955,0.631548


In [149]:
len(torch_mod4.fighter_cols)

31

In [154]:
torch_mod4 = SymmetricModelWrapper(
    fighter_cols=(elo_fighter_cols+real_fighter_cols),
    opponent_cols=(elo_opponent_cols+real_opponent_cols),
    width_1=16,
    width_2=16,
    n_pca=31, # all features, just rotates them to be orthogonal
    lr=0.0005,
    n_epochs=5,
)

tscv.get_cross_val_preds(torch_mod4, torch_df)
torch_metrics4 = tscv.score_preds()
torch_metrics4

training on date range: 2007-06-23 2012-03-02


100%|████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 149.54it/s]


training on date range: 2007-06-23 2015-03-14


100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 91.19it/s]


training on date range: 2007-06-23 2017-10-28


100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 69.44it/s]


training on date range: 2007-06-23 2020-01-25


100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 49.02it/s]


,test_fold,min_test_date,max_test_date,n_test_days,n_test_fights,log_loss,accuracy_score,ml_log_loss
0,1,2012-04-14,2015-03-14,107,458,0.614998,0.661572,0.614366
1,2,2015-03-21,2017-10-28,107,547,0.664981,0.592322,0.661040
2,3,2017-11-04,2020-01-25,107,748,0.641468,0.631016,0.642368
3,4,2020-02-08,2022-03-05,107,886,0.626653,0.635440,0.631548


In [159]:
torch_df5 = torch_df.assign(
    gender_int = torch_df["gender"].map({"M": 0, "W": 1})
)

torch_mod5 = SymmetricModelWrapper(
    fighter_cols=(elo_fighter_cols+real_fighter_cols+["gender_int"]),
    opponent_cols=(elo_opponent_cols+real_opponent_cols+["gender_int"]),
    width_1=32,
    width_2=32,
    n_pca=32, 
    lr=0.0005,
    n_epochs=5,
)

tscv.get_cross_val_preds(torch_mod5, torch_df5)
torch_metrics5 = tscv.score_preds()
torch_metrics5

training on date range: 2007-06-23 2012-03-02


100%|████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 146.00it/s]


training on date range: 2007-06-23 2015-03-14


100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 93.76it/s]


training on date range: 2007-06-23 2017-10-28


100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 65.12it/s]


training on date range: 2007-06-23 2020-01-25


100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 43.49it/s]


,test_fold,min_test_date,max_test_date,n_test_days,n_test_fights,log_loss,accuracy_score,ml_log_loss
0,1,2012-04-14,2015-03-14,107,458,0.617142,0.659389,0.614366
1,2,2015-03-21,2017-10-28,107,547,0.660370,0.594150,0.661040
2,3,2017-11-04,2020-01-25,107,748,0.642618,0.631016,0.642368
3,4,2020-02-08,2022-03-05,107,886,0.623647,0.654628,0.631548
